In [15]:
import numpy as np 
import pandas as pd 
import rasterio
import rasterio.features
import rasterio.warp
import folium
from pyproj import transform, Proj, CRS

from FDSSC import models, utils
from matplotlib import pyplot as plt

In [44]:
datadir = "data"

dirlist = []

for root, dirs, files in os.walk(datadir):
    for name in dirs:
        dirlist.append(root + "/" + name)

dir2files = {}
for name in dirlist:
    for root, dirs, files in os.walk(name):
        namelist = []
        for name in files:
            if "TIF" in name:
                namelist.append(root + "/" + name)
        dir2files[root] = namelist


In [53]:
def collate_images(dirlist):
    for directory in dirlist:
        arrays = {}
        # Read metadata of first file
        with rasterio.open(directory[0]) as src0:
            metadata = src0.meta

        metadata.update(count=len(directory))
        with rasterio.open(("_".join(directory[0].split("/")[-1].split(".")[0].split("_")[1:-1]) + ".TIF"), "w", **metadata) as dest:
            for image in directory:
                with rasterio.open(image) as dset:
                    bnum = int(image.split(".")[0].split("_")[-1].replace("B", ""))
                    dest.write_band(bnum, dset.read(1))

In [54]:
collate_images(list(dir2files.values())[0:1])

In [17]:
dataset = rasterio.open(dir2files.values()[0][0])

In [9]:
# read the csv into a pandas data frame
OCC_df = pd.read_csv('miningSites/processedMS.csv', header=0, index_col=False)

OCC_df = OCC_df.loc[OCC_df["MAJOR_COMMODITY_CODE"] == "Au"]


In [18]:
m = folium.Map(location=[-31.743944369555788, 138.29644592694788], tiles='OpenStreetMap', zoom_start=8)

inproj, outproj = dataset.crs, Proj('EPSG:4326')

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, 
        control = True, opacity=0.7).add_to(m)
image_loc = folium.vector_layers.Polygon([(y, x) for x, y in utils.findshape(dataset)[0]["coordinates"][0]]).add_to(m)
dataset = rasterio.open("data/LC08_L1TP_098083_20200125_20200128/RT_LC08_L1TP_098083_20200125_20200128_01_T1_B1.TIF")
image_loc = folium.vector_layers.Polygon([(y, x) for x, y in utils.findshape(dataset)[0]["coordinates"][0]]).add_to(m)
for i in OCC_df.index:
    folium.Marker([OCC_df.loc[i,'LATITUDE'], 
                    OCC_df.loc[i,'LONGITUDE']]).add_to(m)

m